# Set up

### Import modules

In [19]:
import os

from acrg.BC import timevarying_BC
from acrg.config.paths import Paths

### Define inputs

In [37]:
years = [2010, 2011]
species = ['apo', 'co2']

adjust = {'apo': -18.0, 'co2': 310.052}

# Create boundary conditions

### Find data file

In [38]:
jena_file_names = {spec: {year: f'apo99X_WAO_v2021_mix_{year}.nc' if spec=='apo' else \
                                f's10oc_v2021_mix_{year}.nc'
                          for year in years}
                   for spec in species}
jena_files = {spec: {year: os.path.join(Paths.data, 'Gridded_fluxes', spec.upper(), 'Jena_Carboscope', jena_file_name)
                     for year, jena_file_name in jena_spec.items()}
              for spec, jena_spec in jena_file_names.items()}

for spec, jena_spec in jena_files.items():
    for year, jena_file in jena_spec.items():
        if not os.path.isfile(jena_file):
            print(f'File not found: {jena_file}')
        else:
            print(f'File found: {jena_file}')

File found: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2010.nc
File found: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2011.nc
File found: /group/chemistry/acrg/Gridded_fluxes/CO2/Jena_Carboscope/s10oc_v2021_mix_2010.nc
File found: /group/chemistry/acrg/Gridded_fluxes/CO2/Jena_Carboscope/s10oc_v2021_mix_2011.nc


### Create boundary conditions file

In [39]:
out_path = {spec: os.path.join(Paths.lpdm, 'bc', 'EUROPE', f'{spec}_JenaCarboscope') \
                  if spec in ['co2'] else \
                  os.path.join(Paths.lpdm, 'bc', 'EUROPE')
            for spec in species}

for spec, jena_spec in jena_files.items():
    print(f'Saving {spec} bcs to dir: {out_path[spec]}')

    if not os.path.isdir(out_path[spec]):
        os.mkdir(out_path[spec])

    for year, jena_file in jena_spec.items():
        print(f'{year}, raw file: {jena_file}')
        if os.path.isfile(jena_file):
            bc_obj = timevarying_BC.BoundaryConditions(vmr_var        = f'{spec}mix',
                                                       gph_height_var = 'gph',
                                                       filename       = jena_file,
                                                       time_coord     = 'time',
                                                       species        = spec.upper(),
                                                       domain         = 'EUROPE',
                                                       start_date     = None,
                                                       adjust         = adjust[spec])

            bc_obj.make_bc_file(fp_directory    = None,
                                fp_height_coord = 'height',
                                reverse         = None,
                                convert_units   = False,
                                datasource      = 'JenaCarboscope',
                                out_path        = out_path[spec],
                                glob_attrs      = {},
                                copy_glob_attrs = ['history', 'references'],
                                verbose         = True)
        else:
            print(f'File not found:\n{jena_file}')

Saving apo bcs to dir: /group/chemistry/acrg/LPDM/bc/EUROPE
2010, Raw file: /group/chemistry/acrg/Gridded_fluxes/APO/Jena_Carboscope/apo99X_WAO_v2021_mix_2010.nc
Adjusting vmr by -18.0

Cutting vmr data to edges
-------------------------
Using footprint file: /group/chemistry/acrg/LPDM/fp_NAME/EUROPE/MHD-10magl_UKV_hfo-1234yf_EUROPE_201404.nc to extract domain

Interpolating
-------------
-- Interpolating north boundary vmr --
Interpolating height along lon
Interpolating lat/lon along lon
-- Interpolating south boundary vmr --
Interpolating height along lon
Interpolating lat/lon along lon
-- Interpolating east boundary vmr --
Interpolating height along lat
Interpolating lat/lon along lat
-- Interpolating west boundary vmr --
Interpolating height along lat
Interpolating lat/lon along lat

Saving to netcdf
----------------
Copying attribute history
Copying attribute references
Output filename : apo_EUROPE_201001.nc
Saving boundary conditions to : /group/chemistry/acrg/LPDM/bc/EUROPE/apo_